In [40]:
from openai import OpenAI
import os
import json
from sklearn.metrics.pairwise import cosine_similarity
from dotenv import load_dotenv
load_dotenv()

True

In [49]:
import os
from dotenv import load_dotenv
load_dotenv()

runpod_token = os.getenv("RUNPOD_TOKEN")
# Add this line for debugging to ensure the key is loaded
#print(f"Loaded RunPod Token: {runpod_token}")

client = OpenAI(
    api_key=runpod_token,
    base_url=os.getenv("RUNPOD_CHATBOT_URL"),
)

In [7]:
def get_chatbot_response(client, model_name, messages, temperature=0):
    input_messages = []
    for message in messages:
        input_messages.append({"role":message["role"], "content":message["content"]})

    response = client.chat.completions.create(
        model=model_name,
        messages=input_messages,
        temperature=temperature,
        top_p=0.8,
        max_tokens=2000
    ).choices[0].message.content

    return response

In [8]:
client = OpenAI(
    api_key=os.getenv("RUNPOD_TOKEN"),
    base_url=os.getenv("RUNPOD_CHATBOT_URL"),
)
model_name = os.getenv("MODEL_NAME")

## Get LLM response

In [9]:
messages = [{'role':'user','content':"What's the capital of Italy?"}]
response = get_chatbot_response(client,model_name,messages)
print(response)

The capital of Italy is Rome.


# Prompt Engineering

### Structured Output

In [10]:
system_prompt = """
You are a helpful assistant that answer questions about capitals of countries.

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
[
{
    country: the country that you will get the capital of 
    capital: the capital of the country stated
}
]
"""
messages = [{'role':'system','content':system_prompt}]
print("messages: ", messages)
messages.append({'role':'user','content':"What's the capital of Italy?"})
print("messages: ", messages)
response = get_chatbot_response(client,model_name,messages)
print(response)

messages:  [{'role': 'system', 'content': '\nYou are a helpful assistant that answer questions about capitals of countries.\n\nYour output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:\n[\n{\n    country: the country that you will get the capital of \n    capital: the capital of the country stated\n}\n]\n'}]
messages:  [{'role': 'system', 'content': '\nYou are a helpful assistant that answer questions about capitals of countries.\n\nYour output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:\n[\n{\n    country: the country that you will get the capital of \n    capital: the capital of the country stated\n}\n]\n'}, {'role': 'user', 'content': "What's the capital of Italy?"}]
[
{
    "country": "Italy",
    "capital": "Rome"
}
]


In [11]:
json_response = json.loads(response)
json_response

[{'country': 'Italy', 'capital': 'Rome'}]

### Input Structuring

In [12]:
user_prompt = """
Get me the capitals of the following countries:
```
1. Italy
2. France
3. Germany
``
"""
messages = [{'role':'system','content':system_prompt}]
messages.append({'role':'user','content':user_prompt})
response = get_chatbot_response(client,model_name,messages)
print(response)

[
  {
    "country": "Italy",
    "capital": "Rome"
  },
  {
    "country": "France",
    "capital": "Paris"
  },
  {
    "country": "Germany",
    "capital": "Berlin"
  }
]


In [13]:
json_response = json.loads(response)
json_response

[{'country': 'Italy', 'capital': 'Rome'},
 {'country': 'France', 'capital': 'Paris'},
 {'country': 'Germany', 'capital': 'Berlin'}]

## Chain of Thought

In [14]:
user_prompt = """
Calculate the result of this equation: 1+3

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
{
    result: The final number resulted from calculating the equation above
}
"""
messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client, model_name, messages)
print(response)

{
    "result": 4
}


In [15]:
259/2*8654+91072*33-12971

4113098.0

In [16]:
user_prompt = """
Calculate the result of this equation: 259/2*8654+91072*33-12971

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
{
    result: The final number resulted from calculating the equation above
}
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

{
    "result": 1431449.0
}


In [17]:
4113098.0-1431449.0

2681649.0

In [18]:
user_prompt = """
Calculate the result of this equation: 259/2*8654+91072*33-12971

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
{
    steps: This is where you solve the equation bit by bit following the BEDMAS order of operations. You need to show your work and calculate each step leading to final result. Feel free to write here in free text. 
    result: The final number resulted from calculating the equation above
}
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,model_name,messages)

In [19]:
print(response)

{
    "steps": "First, we need to follow the BEDMAS order of operations. 
    1. Divide 259 by 2: 259/2 = 129.5
    2. Multiply 129.5 by 8654: 129.5 * 8654 = 1121011
    3. Multiply 91072 by 33: 91072 * 33 = 3005016
    4. Add 1121011 and 3005016: 1121011 + 3005016 = 4126027
    5. Subtract 12971 from 4126027: 4126027 - 12971 = 4113056",
    "result": 4113056
}


In [20]:
4113098.0-4113056

42.0

## RAG - Retrieval Augmented Generation

### Asking about a subject that the LLM does not know anything about

In [21]:
user_prompt = """
What's new in iphone 16?
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

Since I'm not aware of any official information about the iPhone 16, I'll provide you with some general information about the latest iPhone models and some rumored features that might be included in future iPhone models.

However, I can tell you about the latest iPhone models, such as the iPhone 14 series, which includes:

1. iPhone 14
2. iPhone 14 Plus
3. iPhone 14 Pro
4. iPhone 14 Pro Max

Some of the key features of the iPhone 14 series include:

1. Improved cameras with a new 48MP main camera on the Pro models
2. Faster A16 Bionic chip
3. Longer battery life
4. New colors and designs
5. Enhanced display with a higher refresh rate

As for the iPhone 16, there's no official information available yet. However, based on rumors and leaks, here are some potential features that might be included:

1. Improved cameras with a new periscope lens or a 3D modeling camera
2. Faster A17 Bionic chip or a new chip design
3. Enhanced display with a higher refresh rate or a new OLED panel
4. New col

### Giving Context to the unknown subject

In [22]:
iphone_16 = """
The iPhone 16 introduces several exciting updates, making it one of Apple's most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.

Powered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.

Apple also introduced advanced audio features like "Audio Mix," which uses machine learning to separate background sounds from speech, allowing for more refined audio capture during video recording. Battery life has been extended, especially in the iPhone 16 Pro Max, which is claimed to have the longest-lasting battery of any iPhone 
9TO5MAC

APPLEMAGAZINE
.

Additionally, Apple has switched to USB-C for faster charging and data transfer, and the Pro models now support up to 2x faster video encoding. The starting prices remain consistent with previous generations, with the iPhone 16 starting at $799, while the Pro models start at $999
"""

In [23]:
user_prompt = f"""
{iphone_16}

What is new in iphone 16?
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

According to the text, the new features and updates in the iPhone 16 include:

1. Larger display sizes:
   - Base model: 6.1-inch display
   - iPhone 16 Plus: 6.7-inch display
   - iPhone 16 Pro: 6.3-inch display
   - iPhone 16 Pro Max: 6.9-inch display

2. Thinner bezels and a more durable Ceramic Shield

3. New A18 chip (A18 Pro for Pro models) for improved performance, with:
   - Enhanced neural engine capabilities
   - Faster GPU for gaming
   - Machine learning tasks

4. Upgraded camera systems:
   - Base iPhone 16: dual-camera setup with a 48MP main sensor
   - Pro models: 48MP Ultra Wide and 5x telephoto camera, with Apple's "Camera Control" button

5. Advanced audio features:
   - "Audio Mix" for refined audio capture during video recording

6. Extended battery life, especially in the iPhone 16 Pro Max

7. Switch to USB-C for faster charging and data transfer

8. Support for up to 2x faster video encoding in Pro models

9. Starting prices remain consistent with previous generat

### Automatically Extract context data from the database

In [24]:
samsung_s23 = """
The Samsung Galaxy S23 brings some incremental but notable upgrades to its predecessor, the Galaxy S22. It features the Snapdragon 8 Gen 2 processor, a powerful chip optimized for the S23 series, delivering enhanced performance, especially for gaming and multitasking. This chip ensures top-tier speed and efficiency across all models, from the base S23 to the larger S23+ and S23 Ultra​
STUFF

TECHRADAR
.

In terms of design, the S23's camera module has been streamlined by removing the raised metal contour around the cameras, creating a cleaner, sleeker look. It also sports the same 6.1-inch 120Hz AMOLED display, protected by tougher Gorilla Glass Victus 2, making it more resistant to scratches and drops​
TECHRADAR
.

The S23 Ultra stands out with its 200MP main camera, offering impressive photo clarity, especially in low-light conditions. The selfie camera across the series has been updated to a 12MP sensor, resulting in sharper selfies. The Ultra model also includes productivity tools such as the S-Pen, which remains an essential feature for note-taking and creative tasks​
STUFF

TECHRADAR
.

Battery life is solid, with the S23 Ultra featuring a 5000mAh battery that lasts comfortably through a day of heavy use. However, charging speeds still lag behind some competitors, with 45W wired charging, which is slower than other brands offering up to 125W charging​
STUFF
.

Overall, the Galaxy S23 series enhances performance, durability, and camera quality, making it a strong contender for users seeking a high-performance flagship.
"""

In [25]:
data = [iphone_16, samsung_s23]

In [26]:
user_prompt = """What is new in iphone 16"""

In [27]:
embedding_client = OpenAI(
    api_key=os.getenv('RUNPOD_TOKEN'),
    base_url=os.getenv('RUNPOD_EMBEDDING_URL')
)

In [33]:
def get_embedding(embedding_client, model_name, text_input):
    output = embedding_client.embeddings.create(input=text_input, model=model_name)

    embeddings = []

    for embedding_object in output.data:
        embeddings.append(embedding_object.embedding)

    return embeddings

In [34]:
output = get_embedding(embedding_client, model_name, user_prompt)

In [29]:
output = embedding_client.embeddings.create(input=user_prompt, model="BAAI/bge-small-en-v1.5")

In [35]:
output

[[-0.05154665187001228,
  -0.03344886377453804,
  0.055910874158144,
  -0.00038864085217937827,
  -0.032746925950050354,
  -0.028642116114497185,
  0.007721316535025835,
  0.03326575085520744,
  -0.0018769208109006286,
  0.03393716737627983,
  0.06714188307523727,
  0.017594225704669952,
  -0.07751835137605667,
  -0.0039789192378520966,
  0.08679614216089249,
  0.024918794631958008,
  0.11499573290348053,
  -0.131842240691185,
  -0.08758963644504547,
  0.021164951846003532,
  -0.00035764495260082185,
  -0.019486404955387115,
  -0.026032738387584686,
  -0.02838270366191864,
  0.030686892569065094,
  0.051882363855838776,
  -0.0024148188531398773,
  -0.021088654175400734,
  -0.01136071141809225,
  -0.16565732657909393,
  -0.005611687898635864,
  -0.008743704296648502,
  0.00839273538440466,
  0.01498484704643488,
  0.00227176072075963,
  -0.03863710165023804,
  -0.00277532497420907,
  0.04156692698597908,
  -0.013870902359485626,
  0.06366270780563354,
  0.025040870532393456,
  0.0906415

In [36]:
user_prompt_embedding = get_embedding(embedding_client, model_name, user_prompt)[0]

In [37]:
data_embedding = [get_embedding(embedding_client, model_name, text)[0] for text in data]

In [38]:
data_embedding

[[-0.0817207545042038,
  0.0334145724773407,
  0.031080130487680435,
  0.0008568206103518605,
  -0.04433915391564369,
  0.01893492415547371,
  0.03161415457725525,
  0.01858399622142315,
  0.04494946822524071,
  0.05462291091680527,
  0.07964569330215454,
  0.007201679516583681,
  -0.06286212056875229,
  -0.008788490667939186,
  0.031064871698617935,
  0.06408274173736572,
  0.13280385732650757,
  -0.16429594159126282,
  -0.0559045635163784,
  0.014395730569958687,
  0.05437878519296646,
  -0.04864185303449631,
  -0.008262096904218197,
  -0.01525016687810421,
  0.040341611951589584,
  0.04696349427103996,
  -0.006206108722835779,
  -0.010329527780413628,
  0.02720464952290058,
  -0.20958109200000763,
  0.008071374148130417,
  0.011389944702386856,
  0.04940474405884743,
  0.005126619245857,
  -0.0521816611289978,
  0.0013693871442228556,
  -0.02683846279978752,
  -0.001562970457598567,
  0.028837233781814575,
  0.04314904659986496,
  -0.00026868024724535644,
  0.06228232383728027,
  -0

In [39]:
len(data_embedding)

2

In [41]:
data_similarity = cosine_similarity([user_prompt_embedding], data_embedding)

In [42]:
data_similarity

array([[0.76157921, 0.53692964]])

In [43]:
closest_entry_index = data_similarity.argmax()
closest_entry_index

np.int64(0)

In [44]:
data[closest_entry_index]

'\nThe iPhone 16 introduces several exciting updates, making it one of Apple\'s most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.\n\nPowered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.\n\nApple also introduced advanced audio features like "Audio Mix," which uses machine l

In [45]:
user_prompt_with_data = f"""
{data[closest_entry_index]}

{user_prompt}
"""

In [47]:
print(user_prompt_with_data)



The iPhone 16 introduces several exciting updates, making it one of Apple's most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.

Powered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.

Apple also introduced advanced audio features like "Audio Mix," which uses machine learnin

In [48]:
messages = [{"role":"user", "content":user_prompt_with_data}]
response = get_chatbot_response(client, model_name, messages)
print(response)

According to the provided information, the new features and updates in the iPhone 16 include:

1. **Larger displays**: 
   - Base model: 6.1-inch display
   - iPhone 16 Plus: 6.7-inch screen
   - iPhone 16 Pro: 6.3-inch display
   - iPhone 16 Pro Max: 6.9-inch display

2. **Thinner bezels**: The iPhone 16 Pro and Pro Max have the thinnest bezels seen on any Apple product so far.

3. **Durable Ceramic Shield**: The iPhone 16 features a more durable Ceramic Shield.

4. **New A18 chip (A18 Pro for Pro models)**: The new chip delivers significant performance improvements, including:
   - Enhanced neural engine capabilities
   - Faster GPU for gaming
   - Machine learning tasks

5. **Upgraded camera systems**:
   - Base iPhone 16: Dual-camera setup with a 48MP main sensor
   - Pro models: 48MP Ultra Wide and 5x telephoto camera
   - Enhanced by Apple’s "Camera Control" button for more flexible photography options

6. **Advanced audio features**: "Audio Mix" uses machine learning to separate